# Guide complet : encodeur 'efficientnet'

Ce carnet décrit en détail l'encodeur EfficientNet 1D d'origine utilisé par PhysAE, ainsi que toutes les options modifiables pour l'entraînement et l'inférence.

## Pré-requis

- Python 3.10 ou ultérieur
- PyTorch (CPU ou GPU) et PyTorch Lightning
- Installation locale du paquet `physae` (`pip install -e .` depuis la racine du dépôt)
- Accès aux fichiers de configuration YAML (`physae/configs/...`)


## Exemple SSH

Utilisez la commande suivante pour vous connecter à un nœud distant et synchroniser votre dépôt avant l'entraînement :

```bash
# Connexion SSH
ssh utilisateur@mon-serveur
# Synchronisation du dépôt
git clone git@github.com:mon-org/physae.git
cd physae
python -m venv .venv
source .venv/bin/activate
pip install -e .[train]
```

## Inspection rapide du registre

Le registre d'encodeurs permet de vérifier que le modèle est disponible :

In [ ]:
from physae.models import available_encoders

for name in available_encoders():
    print('-', name)


## Instanciation de l'encodeur

On peut construire l'encodeur directement via `build_encoder` pour l'utiliser dans un pipeline personnalisé.

In [ ]:
from physae.models import build_encoder

encoder = build_encoder('efficientnet', in_channels=1)
print(encoder.__class__.__name__)
print('Nombre de paramètres:', sum(p.numel() for p in encoder.parameters()))


## Hyperparamètres modifiables

- `width_mult` *(float, défaut 1.0)* : élargit proportionnellement toutes les largeurs de canaux.
- `depth_mult` *(float, défaut 1.0)* : répète davantage chaque bloc MBConv.
- `se_ratio` *(float, défaut 0.25)* : rapport de compression pour les modules squeeze-and-excitation.
- `expand_ratio_scale` *(float, défaut 1.0)* : multiplie le facteur d'expansion de chaque bloc.
- `norm_groups` *(int, défaut 8)* : nombre de groupes pour la normalisation de groupe.
- `block_settings` *(séquence de `MBConvConfig`)* : structure exacte de la pile de blocs.

## Chargement via les configurations YAML

Voici comment recharger l'encodeur dans le modèle complet PhysAE en limitant la taille du jeu de données pour un test rapide.

In [ ]:
from physae.factory import build_data_and_model

small_data_cfg = {
    'n_points': 256,
    'n_train': 128,
    'n_val': 32,
    'batch_size': 8,
}

model_bundle = build_data_and_model(
    n_points=small_data_cfg['n_points'],
    n_train=small_data_cfg['n_train'],
    n_val=small_data_cfg['n_val'],
    batch_size=small_data_cfg['batch_size'],
    config_overrides={
        'model': {
            'encoder': {
                'name': 'efficientnet',
                'params': {
                    'width_mult': 1.0
                }
            }
        }
    }
)

model, (train_loader, val_loader), metadata = model_bundle
print(type(model).__name__)
print('Dimensions des entrées attendues :', metadata['input_shape'])


## Lancement d'un entraînement d'essai

On peut maintenant lancer un entraînement abrégé (quelques itérations) en ajustant l'entraîneur PyTorch Lightning.

In [ ]:
from physae.pipeline import train_stages

train_results = train_stages(
    stages='A',
    max_epochs=1,
    limit_train_batches=5,
    limit_val_batches=2,
    data_overrides={
        'n_train': 128,
        'n_val': 32,
        'batch_size': 8,
    },
    stage_overrides={
        'A': {
            'model': {
                'encoder': {
                    'name': 'efficientnet'
                }
            }
        }
    },
    trainer_kwargs={
        'accelerator': 'cpu',
        'devices': 1,
        'max_epochs': 1,
    }
)

print(train_results.metrics['A'])


## Personnalisation avancée

L'encodeur renvoie un tenseur de caractéristiques et peut être couplé à n'importe quel raffineur enregistré. Les paramètres supplémentaires fournis à `build_encoder` sont transmis directement au constructeur `EfficientNetEncoder`.

### Exemple de modification du YAML

```yaml
model:
  encoder:
    name: efficientnet
    params:
      width_mult: 0.75
      depth_mult: 1.2
      norm_groups: 4
```

Enregistrez ce fragment dans un fichier puis passez-le à `--stage-overrides` ou fusionnez-le via `config_overrides`.

## Inférence et export

Une fois le modèle entraîné, utilisez `physae.cli` pour effectuer une inférence ou exporter les prédictions :

```bash
physae optimise --stages A --n-trials 5 --metric val_loss --output-dir studies
physae train --stages A --studies-dir studies --devices 1 --accelerator cpu
physae predict --checkpoint chemin/vers/last.ckpt --output predictions.parquet
```

## Aller plus loin

- Tester différents jeux d'hyperparamètres avec Optuna (`physae.optimization`).
- Modifier `train_ranges` et `noise` dans `physae/configs/data`.
- Brancher un raffineur personnalisé via `register_refiner`.